In [ ]:
# !pip install scipy
# !pip install pysindy plotly

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import solve_ivp

# Parameters for the Lorenz system
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Lorenz system equations
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Initial conditions and integration time
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 149)  # Time interval
t_eval = np.linspace(t_span[0], t_span[1], 10000)  # Time points

# Solve the Lorenz system
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)

# Extract data
x = sol.y[0]  # Time series x(t)

# Choose delay and embedding dimension
# tau = 13  # Delay in time steps (chosen number, can be experimented with)
embedding_dim = 3  # Embedding dimension

# function to create delay vectors
def delay_embedding(data, delay, dim):
    n = len(data)
    max_delay = (dim - 1) * delay
    embedded_data = np.zeros((n - max_delay, dim))
    for i in range(dim):
        embedded_data[:, i] = data[delay * i:n - max_delay + delay * i]
    return embedded_data

# def delay_embedding(data, delay, dim):
#     n = len(data)
#     embedded_data = np.zeros((n - (dim - 1) * delay, dim))
#     embedded_data[:, 0] = data[0:n - (dim - 1) * delay]
#     embedded_data[:, 1] = data[delay:n - (dim - 1 - 1) * delay]
#     embedded_data[:, 2] = data[2 * delay:n - (dim - 1 - 2) * delay]
#     return embedded_data

def delay_embedding2(data, delay1, delay2, dim):
    n = len(data)
    embedded_data = np.zeros((n - delay1 - delay2, dim))
    embedded_data[:, 0] = data[0:n - delay1 - delay2]
    embedded_data[:, 1] = data[delay1:n - delay2]
    embedded_data[:, 2] = data[delay1 + delay2:n]
    return embedded_data

def test_for_tau(tau):
    # Create delay vectors
    embedded_data = delay_embedding(x, tau, embedding_dim)
    # Create an interactive Plotly graph
    fig = go.Figure()
    # Add graph for tau
    fig.add_trace(go.Scatter3d(
        x=embedded_data[:, 0],
        y=embedded_data[:, 1],
        z=embedded_data[:, 2],
        mode='lines',
        name=f'tau = {tau}',
        line=dict(color='blue', width=2)
    ))
    fig.update_layout(
        scene=dict(
            xaxis_title='x',
            yaxis_title=f'y',
            zaxis_title=f'z'
        ),
        title=f'tau = {tau}',
        width=800,
        height=600,
        margin=dict(l=0, r=0, b=0, t=40)
    )
    # Display the graph
    fig.show()

for tau in range(7, 21, 6):
    test_for_tau(tau)
# embedded_data = delay_embedding2(x, 100, 500, embedding_dim)

# fig.add_trace(go.Scatter3d(
#     x=embedded_data[:, 0]+9,
#     y=embedded_data[:, 1]+0,
#     z=embedded_data[:, 2]+9,
#     mode='lines',
#     name=f'2',
#     line=dict(color='red', width=2)
# ))



In [ ]:
# Create an interactive Plotly graph
fig = go.Figure(data=[go.Scatter3d(
    x=sol.y[0],
    y=sol.y[1],
    z=sol.y[2],
    mode='lines',
    line=dict(color='blue', width=2)
)])

# Configure the layout of the graph
fig.update_layout(
    title="Original Lorenz Attractor",
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the graph
fig.show()